<h1>Das Beispiel-Netzwerk</h1><br>
Das zu lösende Probleme soll die Einordnung von 28px mal 28px großen Bildern von Zahlen in Schreibschrift in die Kategorien 0 bis 9 sein.

In [2]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

print(train_labels[0])
print(train_images[0])

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
5
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   

Nun wird ein Netzwerk gebaut, dem die Bilder train_images und die Zuordnung train_labels. Daraus entwickelt das Netzwerk Zuordnungen, die anhand des Vergleichs des errechneten Outputs von test_images[i] mit test_labels[i] verglichen werden.

In [3]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation = 'relu', input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation = 'softmax'))

Wir bauen ein Netzwerk, das in der ersten Schicht 512 Einheiten besitzt und Daten der Größe <code>(28 * 28, )</code> erwartet. <code>Dense</code> heißt, dass jede Einheit dieser Schicht mit allen Einheiten der vorhergehenden Schicht verbunden ist. Die letzte Schicht besitzt eine <code>softmax</code> Aktivierungsfunktion, was bedeutet, dass sie 10 Wahrscheinlichkeiten für die 10 Kategorien zurückgeben wird.<br>
Nun müssen noch drei Dinge definiert werden, bevor das Netzwerk die Arbeite aufnehmen kann:
1. Die Loss-Function: wie das Netzwerk Erfolg beim Vergleich mit den Testdaten misst
2. Der Optimierer, also wie sich das Netzwerk optimiert, basierend auf der Loss-Function und den Daten
3. Die zu beobachtenden Metriken während des Trainings


In [4]:
network.compile(
    optimizer = 'rmsprop',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Bevor das Netzwerk trainiert werden kann, müssen die Daten in Vektoren aus Nullen und Einsen umgewandelt werden. Unsere Trainingsbilder haben die Form <code>(60000, 28, 28)</code>, also 60000 Bilder mit 28 mal 28 Pixeln, wobei jeder Pixel durch eine Zahl zwischen 0 und 255 dargestellt wird, also wie schwarz/weiß dieser ist. Diese werden nun in <code>float32</code>-Werte <i>zwischen</i> 0 und 1 umgewandelt in der Form <code>(60000, 28 * 28)</code> (60000 Beobachtungen mit einem 1D-Array aus Werten zwischen 0 und 1).

In [5]:
train_images = train_images.reshape((60000, 28 * 28)) # neue Form mit flachen Arrays
train_images = train_images.astype('float32') / 255 # float32 auf 0...1 normiert

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# labels umwandeln
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# alles ist getan, das Netzwerk kann traniert werden
network.fit(
    train_images, train_labels,
    epochs = 5,
    batch_size = 128
)

test_loss, test_acc = network.evaluate(test_images, test_labels)
print('Accuracy:', test_acc)

Epoch 1/5
60000/60000 [==============================] - 3s 43us/step - loss: 0.2544 - acc: 0.9268
Epoch 2/5
60000/60000 [==============================] - 2s 39us/step - loss: 0.1033 - acc: 0.9696
Epoch 3/5
60000/60000 [==============================] - 2s 39us/step - loss: 0.0683 - acc: 0.9795
Epoch 4/5
60000/60000 [==============================] - 2s 39us/step - loss: 0.0503 - acc: 0.9848
Epoch 5/5
10000/10000 [==============================] - 0s 28us/step
Accuracy: 0.9802


<h1>Datenrepräsentationen</h1><br>
Tensoren sind die generelle Bezeichnung für Vektoren, Matrizen, Skalare, etc. Sie sind die in NN Netzwerk am meisten genutzte Datenrepräsentation.<br><br>
<b>Skalare - 0D</b><br>
Das sind Tensoren mit nur einer Zahl. In Numpy können float32 oder float64-Zahlen Skalare sein.<br><br>
<b>Vektoren - 1D</b><br>
Vektoren haben exakt eine Dimension, entlang der Zahlen angeordnet sind.<br><br>
<b>Matrizen - 2D</b><br>
Haben zwei Dimensionen, genannt Zeilen und Spalten. Beispiele für Daten aus der realen Welt, die als Matrix angeordnet sind, sind Daten, bei den für n Beobachtungen p Variablen augezeichnet worden sind: <code>(Beobachtungen, Variablen)</code>.<br><br>
<b>3+D</b><br>
3D-Tensoren kann man z.B. als Anordnung in einem Würfel interpretieren. Ein Beispiel für 3D-Daten sind Zeitreihendaten, wo wieder für n Beobachtungen p Variablen aufgezeichnet worden sind, dies aber zu T Zeitpunkten wiederholt wurde: <code>(Beobachtungen, Variablen, Zeitpunkte)</code>.<br>
4D - Bilder: <code>(Beobachtungen, Px in x-Dimension, Px in y-Dimension, Kanäle (Farben))</code>.<br>
5D - Videos: <code>Beobachtungen, Frame, Px in x-Dimension, Px in y-Dimension, Kanäle (Farben))</code>

In [6]:
import numpy as np
x = np.array(12)
print(x) # Skalar
print(x.ndim) # so kann man sich zeigen lassen, wie viele Dimensionen Tensor hat

print(np.array([12, 5, 2])) # Vektor
x = np.array([
    [1, 2, 4],
    [5, 2, 5],
    [6, 9, 2]
]).astype('float32')
print(x) # Matrix

12
0
[12  5  2]
[[1. 2. 4.]
 [5. 2. 5.]
 [6. 9. 2.]]


<h1>Manipulation von NumPy-Datentypen</h1><br>
Tensoren haben folgende wichtige Eigenschaften:

In [7]:
print(x.ndim) # Zahl der Achsen
print(x.shape) # wie viele Einheiten entlang jeder Achse: hier 3 Zeilen, 3 Spalten
print(x.dtype) # der NumPy-Datentyp

print(train_images.ndim)
print(train_images.shape)
print(train_images.dtype)


2
(3, 3)
float32
2
(60000, 784)
float32


<b>Auswählen von Daten aus einem Tensor</b><br>

In [8]:
mySlice = train_images[10:100] # Auswahl Beobachtungen 1 bis 99
print(mySlice.shape)
# ist das gleiche wie
mySlice = train_images[10:100, 0:28, 0:28]
print(mySlice.shape)

# Auswahl der unteren rechten Ecke ALLER Bilder:
mySlice = train_images[:, 14:, 14:] # alle Bilder, Px 14 und darüber, px 14 und darüber


(90, 784)


IndexError: too many indices for array

<b>Batches von Daten</b><br>
In allen Datensätzen ist der nullte Dimension normalerweise die Dimension der jeweiligen Beobachtung. Diese wird beim Tranining des Modells in Batches aufgeteilt, hier in Batches von 128 Beobachtungen:

In [13]:
batch = train_images[:128] # erster Batch Groesse 128 von 0 bis 127
batch = train_images[128:256] # zweiter 128 ... 255
# batch = train_images[128 * n:128 * (n + 1)]

<b>Tensoroperationen</b><br>
Neuronale Netzwerke können auf eine Reihe von Tensoroperationen (bspw. Matrixmultiplikation) reduziert werden, die folgend erklärt werden.

In [14]:
from keras import layers
exampleLayer= layers.Dense(512, activation = 'relu')

Diese Schicht kann als eine Funktion interpretiert werden, die einen 2D-Tensor als Input nimmt und ebenso einen 2D-Tensor wieder ausgibt. 
$$output = relu(dot(W, input) + bias)$$ mit $$relu(x) = max \{x, 0\}$$
All die Operationen sind Operationen, die auf allen Elementen eines Tensors ausgeführt wird: 

In [15]:
import numpy as np
x = np.array([1, 2, 3]).astype('float32')
y = np.array([-2, 3, -4]).astype('float32')
z = x + y # elementweise Addition
print(z)
print(np.maximum(z, 0.)) # elementweise relu-Funktion

[-1.  5. -1.]
[0. 5. 0.]


All diese Operationen auf einzelnen Elementen sind durch Basic Linear Algebra Subprograms (BLAS) unglaublich schnell. Das sind Subroutinen, geschrieben in Fortran oder C, die installiert werden sollten, um all diese Opeartionen schneller zu machen.

<b>Broadcasting</b><br>
Es sollte für NumPy noch das Prinzip des Broadcasting erwähnt werden, bei dem Tensoren erweitert werden, sodass Operationen mit anderen Tensoren möglich sind, die ohne Erweiterung mathematisch nicht möglich wären. Man kann sich die Implementierung so vorstellen:

In [16]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2 # x ist eine Matrix, assert heisst: wenn nicht wahr, schmeisse Error
    assert len(y.shape) == 1 # y ist ein Vektor
    assert x.shape[1] == y.shape[0] # eine Spalte in x ist so lang wie der y-Vektor
    x = x.copy() # überschreibe nicht die originale Variable
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j] # also zu jeder Spalte von x wird y addiert
            # man kann sagen, y wird solange neben sich selbst gelegt, bis es so gross wie x ist und dann koennen sie addiert werden
    return x

Das Matrix-Produkt kann folgendermaßen ausgedrückt werden:

In [17]:
import numpy as np
x = np.array([
    [1, 2, 3],
    [2, 3, 1],
    [3, 2, 1]
]).astype('float32')

y = np.array([
    [1, 2],
    [2, 3],
    [3, 2]
]).astype('float32')

print(x.shape)
print(y.shape)

z = np.dot(x, y)
print(z)
print(z.shape)

(3, 3)
(3, 2)
[[14. 14.]
 [11. 15.]
 [10. 14.]]
(3, 2)


<b>Umformung von Tensoren</b><br>
Das beudeutet, die Spalten und Zeilen eines Tensors so umzuformen, dass sie eine bestimmte Zielform erreichen.

In [18]:
x = np.array([
    [1, 2],
    [2, 3],
    [3, 2]
]).astype('float32')
print(x.shape)

x = x.reshape((6, 1))
print(x.shape)
print(x)

x = x.reshape((2, 3))
print(x)

# besondere Form: transponieren
print(np.transpose(x))

(3, 2)
(6, 1)
[[1.]
 [2.]
 [2.]
 [3.]
 [3.]
 [2.]]
[[1. 2. 2.]
 [3. 3. 2.]]
[[1. 3.]
 [2. 3.]
 [2. 2.]]


<h1>Optimierung und eine mathematischere Sichtweise</h1><br>
Für bereits erwähnt, kann jede Schicht eines NN ausgedrückt werden als <code>output = relu(dot(W, input) + bias)</code>. Dabei sind W und bias sogenannte trainierbare Parameter, sie repräsentieren das Wissen des NN und werden in einem Schritt genannt <i>zufällige Initialisierung</i> mit Werten belegt, die den Startpunkt für die Optimierung bieten. Nun werden diese Parameter langsam angepasst, abhängig von einem Feedback-Signal. Diese Phase wird das Training des NN genannt. Das passiert in der Training-Loop:
1. Hole dir ein Batch von Input samples X und den dazugehörigen Labels y
2. Gib X in das NN und erhalte die vorhergesagten Werte y_pred
3. Errechne den Verlust, also den Unterschied von y_pred und y
4. Aktualisiere die Parameter, sodass der Verlust ein wenig gemindert wird<br>

Der schwierige Teil ist Teil 4: wie genau sollen die Gewichte angepasst werden? Alle Funktionen, die im NN genutzt werden sind differenzierbar.<br>
Stell dir einen Input-Vektor <code>x</code>, eine Matrix <code>W</code>, Labels <code>y</code> und eine Loss-Funktion <code>loss</code> vor. Wir nutzen <code>W</code> und <code>x</code>, um <code>y_pred</code> zu errechnen: <code>y_pred = dot(W, x)</code>. Dann folgt der Vergleich mit den Labels: <code>loss_value = loss(y_pred, y)</code>.<br>
Werden <code>x</code> und <code>y</code> nun eingefroren, kann das System als eine Funktion interpretiert, die die Gewichte auf den Verlust mappt: <code>loss_value = f(W)</code>. Nun wird der Gradient <code>gradient(f)(W)</code> in <code>W = W0</code> ermittelt. Sie stellt die Steigung der Loss-Function dar. Nun muss W mit einem kleinem inkrementellen Schritt <code>step</code> modifiziert werden (<code>W1 = W0 - step * gradient(f)(W0)</code>). Das heißt der Gradient in W0 wird mit dem negativen step multipliziert und dann von W0 abgezogen: <b>jedes Element in W wird ein wenig in die Richtung verändert, die den Loss fallen lässt</b>.
<img src="./imgs/Loss.JPG">

<h1>Der Backpropagation Algorithmus</h1><br>
Backpropagation beschreibt die Anpassung der Gewichte und Biases als Folge des Wertes der Verlustfunktion nach einem Trainingsbeispiel. Wie bereits erwähnt wollen wir die Gewichte des Netzwerks so verändern, dass sie auf ein (lokales) Minimum der Kostenfunktion zusteuern, also in Richtung $-\nabla{C(w_{1}, ..., w_{n})}$, wobei die Kostenfunktion $C$ ein Funktion des Unteschieds zwischen dem Output des NN und dem richtigen Wert ist. Kleiner Einschub: der Gradient $\nabla$ zeigt, in welche Richtung sich die Funktion nach oben bewegt, also zeigt $-\nabla$, in welche Richtung sie sich nach unten bewegt. Der Wert zeigt also, in welche Richtung alle Gewichte gestoßen werden müssen, damit sich der Output dem Ziel angleicht. Der Backpropagation-Algorithmus errechnet diesen komplizierten Gradienten.<br><br>
<b>Quelle für die Bilder aus dem folgenden Beispiel</b>: https://www.youtube.com/watch?v=Ilg3gGewQ5U.
<img src="./imgs/example_NN_3blue1brown.png">
Wir haben ein untrainiertes Netzwerk, also ist der Unterschied zwischen dem Ziel "2" und der Outputschicht noch groß, wie im Bild zu sehen. <img src="./imgs/updown_3blue1brown.png"> Wie bringen wir die Gewichte richtig dazu, dass das Outputwert, der 2 indiziert erhöht wird, und alle anderen gesenkt werden? Um das möglichst effizient zu tun, sollte die Größe der Veränderung proportional dazu sein, wie unterschiedlich sie zum gewünschten Ergebnis sind.<br>
Wir wissen nun, wie sich der Output verändern sollte und wir können das tun, indem wir die Gewichte zu dieser Einheit verändern. Nehmen wir also Beispiel das Outputneuron für den Wert 2, dessen Wert wir erhöhen wollen. Zur Erinnerung, der Wert für die 2 entsteht aus ihren Gewichten mal der Aktivierung der Neuronen in der vorigen Schicht plus einem Bias, dann durch eine Aktivierungsfunktion $f$: $$a_{2}=f(w_{0,2}a_{0,2}+...+w_{n-1,2}a_{n-1,2}+b)$$<img src="./imgs/weightsTo2_3blue1brown.png">
Wie geben wir dem Outputneuron für die 2 ein stärkeres Signal?<br>
Erstens, wir verändern die Gewichte zu diesem Neuron: wir verstärken jene Gewichte von Neuronen, die bei der zwei aktiviert worden. Wir verändern $w_i$ im Verhältnis zu $a_i$, da Veränderungen in stark aktivierten Neuronen der vorigen Schicht das Signal des 2-Outputneurons schneller erhöhen als weniger aktivierte. Gleichzeitig erniedrigen wir das Gewicht der wenig aktivierten Neuronen.<br>
Zweitens, wir verändern die Aktivierungen der vorigen Schicht. Wenn alle Neuronen mit einem starken Gewicht zum 2-Neuron stärker werden und alle mit einem kleinen/negativen Gewicht schwächer, dann wird das 2-Neuron eher aktiviert werden. Wir ändern also $a_i$ im Verhältnis zu $w_i$.<br>
Natürlich können wir die Aktivierung nicht direkt beeinflussen, da das vom Trainingsbeispiel abhängt, <b>aber</b> wir wissen, welche Veränderung in der Aktivierung des Neurons wir uns wünschen. Somit können wir die Gewichte dieses Neurons in der vorigen Schicht anpassen, damit uns die Aktivierung für Schicht vor dem 2-Neuron für das 2-Neuron gelegen kommt. <b>Merke</b>: dieser Prozess setzt sich also bis zu den Gewichten von den Inputneuronen fort: es erfoglte eine <i>backpropagation</i> vom Unterschied im Output bis zu den Gewichten der Inputneuronen.<br>
Außerdem ist es wichtig, dass all das für die Veränderung der Neuron für die Änderung <i>eines</i> Outputneurons ist. Alle Outputneuronen melden also über Backpropagation ein gewünschte Änderung, die aufaddiert wird. <i>Und das alles nur für ein Trainingsbeispiel!</i><img src="./imgs/alleNeuronen_3blue1brown.png"><br><br>
<b>Batching</b><br>
Das Trainieren eines Batches (= einem Teil des Datensatzes) ist gleichzusetzen mit dem Errechnen des Outputs und der Veränderung der Gewichte. Also werden die Outputs für einen Batch errechnet, und anschließend die Gewichte gemäß des "durchschnittlichen Wunsches" zur Veränderung und der damit verbundenen geänderten Position auf $C$.<br>
Je größer der Batch, desto genauer der Schritt in Richtung eines Minimus auf $C$, weil es den Gesamtdatensatz besser repräsentiert. Allerdings wird die Berechnung länger dauern und mehr RAM brauchen.
Betrachtet man die beiden Extreme, ein Batch mit je einem Beispiel und ein Batch mit allen Trainingsbeispielen, dann "wandert" NN mit kleineren Batchgrößen zu ziellos auf $C$ herum, während eine zu große Batch-Größe nicht mehr aus einem lokalen Minimum auf $C$ herausfindet.<br>
<b>Kurz: je größer der Batch, desto genauer die Schritte auf $C$, aber desto mehr Rechenkraft benötigt.</b>